<a href="https://colab.research.google.com/github/mahdijafari78/MovieLens_SVD_Recommender/blob/main/MovieLens_SVD_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install surprise

In [5]:
!pip install numpy==1.23.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 95.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
xarray 2025.3.1 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
bigframes 2.4.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
blosc2 3.3.2 requires numpy>=1.26, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is inco

In [27]:
from surprise import Dataset
from surprise import Reader

data = Dataset.load_builtin('ml-100k')


In [32]:
from surprise.model_selection import GridSearchCV
from surprise import SVD

param_grid = {
    'n_epochs': [10, 20, 30],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.02, 0.1],
    'n_factors': [50, 100, 150]
}

model = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)

In [35]:
#from surprise.model_selection import train_test_split
#trainset, testset = train_test_split(data, test_size=0.25)

In [37]:
model.fit(data)

In [38]:
best_model = model.best_estimator['rmse']

In [39]:
from surprise.model_selection import cross_validate

result = cross_validate(best_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9208  0.9130  0.9026  0.9182  0.9052  0.9120  0.0071  
MAE (testset)     0.7282  0.7224  0.7126  0.7271  0.7155  0.7212  0.0062  
Fit time          3.59    2.58    2.66    2.64    3.60    3.01    0.48    
Test time         0.14    0.14    0.35    0.12    0.12    0.17    0.09    


In [40]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25)

In [41]:
best_model.fit(trainset)

In [43]:
predictions = best_model.test(testset)


In [47]:
for pred in predictions[6:15]:
    print(f"User: {pred.uid}, Item: {pred.iid}")
    print(f"Real rating: {pred.r_ui}, Predicted rating: {pred.est:.2f}")
    print("-----")


User: 85, Item: 1166
Real rating: 4.0, Predicted rating: 3.03
-----
User: 445, Item: 405
Real rating: 4.0, Predicted rating: 1.93
-----
User: 184, Item: 259
Real rating: 3.0, Predicted rating: 2.54
-----
User: 676, Item: 879
Real rating: 3.0, Predicted rating: 3.54
-----
User: 38, Item: 1028
Real rating: 5.0, Predicted rating: 3.77
-----
User: 13, Item: 90
Real rating: 3.0, Predicted rating: 2.75
-----
User: 51, Item: 496
Real rating: 4.0, Predicted rating: 3.81
-----
User: 409, Item: 154
Real rating: 5.0, Predicted rating: 3.47
-----
User: 548, Item: 659
Real rating: 4.0, Predicted rating: 4.26
-----


In [48]:
import pickle

with open("svd_model.pkl", "wb") as f:
    pickle.dump(model, f)
